# B3 index(IFIX)

<img src="https://logospng.org/download/b3/logo-b3-brasil-bolsa-balcao-256.png"  style="float:left;position:absolute; top:10px; border:0px solid red; width:150px"/>
<img src="https://i.pinimg.com/originals/dc/8a/02/dc8a0226bd44d814ff526341e88f3d51.png"  style="float:right; width:150px"/>



### Introduction

The IFIX is the result of a theoretical portfolio of assets, prepared in accordance with the criteria established in this methodology.

B3 indices use procedures and rules contained in the B3 Indices Definitions and Procedures Manual.

**objective**
The purpose of the IFIX is to be an indicator of the average performance of the prices of real estate funds traded on B3's stock exchange and organized over-the-counter markets.

**index type**
The IFIX is a total return index (see B3 Index Definitions and Procedures Manual).

E**ligible assets**
The IFIX comprises the shares of Real Estate Investment Funds listed on B3's stock exchange and over-the-counter markets.

[B3 source](http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-de-fundos-de-investimentos-imobiliarios-ifix.htm)

### Approach

My focus here will be on visualizations to understand better this index and answer the following questions:

<img src='https://images-2.wuaki.tv/system/shots/197968/original/snapshot-1590666404.jpeg' style="float:right; width:250px" />

- How the index behave in important events in the local and international?
- How are daily returns compared to average monthly returns?
- Daily return - IFIX x CDI x IPCA X SELIC x IGP-M ?


#### Setup and load datasets

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.graphics.gofplots import qqplot
from IPython import display
import warnings
warnings.filterwarnings('ignore')
def clear():
    display.clear_output(wait=True)
    display.clear_output()    
clear()

import requests
from datetime import datetime

class  IFIX:
    
    def __init__(self):
        self.indices = self.createIndices()
        self.historical = self.createHistorical()
        
    def  remove_outliers(self, data):
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        data = np.where(data>(Q3+1.5*IQR),(Q3+1.5*IQR),data)
        data = np.where(data<(Q1-1.5*IQR),(Q1-1.5*IQR),data)
        return data
    
    def create_indicators(self, data):
        #Fibonacci 0,1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,
        #Cacl o MACD e sinal
        #short - Exponetial Moving Avage - EMA 
        short_EMA = data.close.ewm(span=30,adjust=False).mean()
        #long - Exponetial Moving Avage - EMA
        long_EMA = data.close.ewm(span=60,adjust=False).mean()
        #MACD line
        macd = short_EMA - long_EMA
        #MACD signal
        signal = macd.ewm(span=5,adjust=False).mean()
        data['monthly_ma'] = short_EMA
        data['semiannually_ma'] = long_EMA
        data['macd'] = macd
        data['signal'] = signal
        media = data['return'].rolling(30).mean()
        media = media.fillna(0)
        data['monthly_ma'] = media
        return data
    
    def createHistorical(self):
        ifix = pd.read_csv('../input/d/franckepeixoto/ifixhistorical/IFIX.SA.csv',squeeze=True, keep_default_na=False)
        ifix.close = ifix.close.apply(lambda r: str(r).replace(",",".")).astype(np.float32)
        ifix.open = ifix.open.apply(lambda r: str(r).replace(",",".")).astype(np.float32)
        ifix['return'] = ifix['return'].apply(lambda r: str(r).replace(",",".")).astype(np.float32)
        ifix['data'] = ifix['data'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
        return self.create_indicators(ifix)
    
    def createIndices(self):
        url = 'https://raw.githubusercontent.com/franckepeixoto/franckepeixoto.github.io/master/indices.json'
        response = requests.get(url)
        indices = response.json()
        nomesSeries  = ['date']+indices["nomesSeries"]
        retorno_diario_indices = pd.DataFrame(columns=nomesSeries)
        columns  = indices["nomesSeries"]
        #retorno_diario_indices 

        data_list = []
        series_list = []
        count = len(indices["datasGrafico"])
        for index in  range(count):
            data = indices["datasGrafico"][index]["data"]
            data = datetime.fromtimestamp(float(data/1000)).strftime('%d/%m/%Y')
            serie = np.array(indices["datasGrafico"][index]["series"])
            data_list.append(data)
            series_list.append(serie)
        retorno_diario_indices['date'] = data_list
        retorno_diario_indices[columns] = series_list
        retorno_diario_indices[columns] = retorno_diario_indices[columns].astype(np.float32)
        retorno_diario_indices['date'] = retorno_diario_indices['date'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
        return retorno_diario_indices
ifix = IFIX()

<img src="https://i.pinimg.com/originals/1e/93/e9/1e93e90925ca7620397c4cd80442e125.png"  style="float:right; width:110px"/>

The Real Estate Investment Funds Index ([IFIX](http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-fundos-de-investimentos-imobiliarios-ifix-composicao-da-carteira.htm)) was launched on **September 3, 2012** by the former BM&F Bovepa, now [B3](http://www.b3.com.br).

Like other stock market indices, it measures the performance of a theoretical asset portfolio, which is made up of Real Estate Investment Fund (**FII**) shares, which are mandatory in B3's stock and over-the-counter markets.

Although its debut date was in **2012**, the index's profitability calculation incorporates values since **December 30, 2010**, providing a greater comparative horizon for investors in the variable income capital market.

In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,5))
sns.lineplot(x=ifix.historical['data'] , linestyle="-", linewidth=3, y=np.log(ifix.historical['close']), color='blue')
clear()
plt.grid(True)
plt.show()

### Exploratory Data Analysis

#### How IFIX index reacted to important moments on the economy and politics?


In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,5))
sns.lineplot(x=ifix.historical['data'] ,linestyle="-", linewidth=3, y=np.log(ifix.historical['close'])             , color='blue')

plt.title('How IFIX index reacted to important moments on the economy and politics?', size=16)
plt.xlabel('Years', size=16)
plt.ylabel('Price', size=16)
plt.xticks(size=14)
plt.yticks(size=14)
setup = dict(size = 21, color = "black")

plt.text("2014-02-14", np.log(1000),'Brazil faces a war against pessimism in the economy', **setup)
plt.axvline("2014-02-14", color='red'
            , linestyle="-"
            , linewidth=3
            , label='Brazil faces a war against pessimism in the economy')


plt.text("2016-06-01", np.log(3000),'Impeachment Dilma Rousseff', **setup)
plt.axvline("2016-06-01", color='green'
            , linestyle="-"
            , linewidth=3
            , label='Impeachment Dilma Rousseff')

plt.text("2020-01-31", np.log(2000),'Brazil approaches 160,000 deaths from coronaviruses', **setup)
plt.axvline("2020-01-31", color='orange'
            , linestyle="-"
            , linewidth=3
            , label='Brazil approaches 160,000 deaths from coronaviruses')


clear()
plt.grid(True)
plt.show()

#### Daily returns vs Average monthly returns

Let's take a look into daily returns and compare then with the average monthly returns to understand how great falls or ups are mitigated when we compare with averages. 

In [ ]:
ifix.historical.head()

In [ ]:
# defyning the plotsize
plt.figure(figsize=(10,7))

# creating the plot to returns
sns.lineplot(x='data'
             ,y='return'
             ,data=ifix.historical
             ,label='daily returns'
            ,color='lightgreen')

# creating the plot to average returns
sns.lineplot(x='data'
             ,y='monthly_ma'
             ,data=ifix.historical
             ,label='monthly average returns'
            ,color='blue')

# setting title, labels and ticks parameters
plt.title('IFIX index returns over the years', size=14)
plt.xlabel('Years', size=14)
plt.ylabel('Returns', size=14)
plt.xticks(size=15)
plt.yticks(size=15)

# setting legend parameters
plt.legend(loc = "lower left"
           , frameon = True
           , fontsize = 15
           , ncol = 2 
           , fancybox = True
           , framealpha = 0.95
           , shadow = True
           , borderpad = 1);
plt.grid(True)

In [ ]:
returns_by_year = pd.DataFrame(round(ifix.historical['return'].groupby(ifix.historical.data.dt.year).sum()*100,2))
returns_by_year['pos_neg'] = returns_by_year>0

plt.figure(figsize=(20,4))
returns_by_year["return"].plot(kind='bar'
                      , color=returns_by_year.pos_neg.map({True: 'forestgreen', False: 'red'})
                      , label='return')

average_returns = np.mean(returns_by_year['return'])

plt.axhline(y=average_returns, color='blue', linestyle='-', label='Average returns')

plt.title('Acumulated returns by year vs average returns',size=14)
plt.xlabel('Years',size=14)
plt.ylabel('Acumulated in %',size=14)
plt.yticks(np.arange(-40,170,step=10), size=15)
plt.xticks(rotation=45, size=15)

plt.legend(loc = "upper right"
           , frameon = True
           , fontsize = 13
           , ncol = 2 
           , fancybox = True
           , framealpha = 0.95
           , shadow = True
           , borderpad = 1);
plt.grid(True)

In [ ]:
data = returns_by_year['pos_neg'].value_counts().values.tolist()
pos_neg = pd.DataFrame(data=data,columns=['counts'],index=['positive','negative'])

# difyning plot size
plt.figure(figsize=(10,6))

 # here i use .value_counts() to count the frequency that each category occurs of dataset
pos_neg['counts'].plot(kind='pie'
                       , colors=['lightgreen','lightgray']
                       , autopct='%1.1f%%' # adding percentagens
                       , shadow=True
                       , startangle=140)

# defyning title and legend parameters
plt.title("What's the proportion of positive annual returns?",size=14)
plt.legend(loc = "upper right"
           , frameon = True
           , fontsize = 14
           , ncol = 2 
           , fancybox = True
           , framealpha = 0.95
           , shadow = True
           , borderpad = 1);

#### SELIC

<IMG SRC='https://upload.wikimedia.org/wikipedia/commons/b/b5/Banco_Central_do_Brasil_logo.png'  STYLE='float:right; width:100px'/>


Selic is the economy's basic interest rate. It is the main instrument of monetary policy used by the **[Central Bank (BC)](https://www.bcb.gov.br/controleinflacao/taxaselic)** to control an issue. It influences all interest rates in the country, such as interest rates on loans, financing and financial investments.



In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,5))
sns.lineplot(x=ifix.indices['date'] , linestyle="-", linewidth=10, y=(ifix.indices['SELIC']), color='blue')
clear()
plt.grid(True)
plt.show()

In [ ]:
ifix.indices[['date','SELIC']].tail()

#### IPCA  

<IMG SRC='https://s.concursosnobrasil.com.br/media/cache/f/concurso-ibge-2021-saiba-tudo-sobre-o-cargo-de-recenseador-960x540.jpg'  STYLE='float:right; width:100px'/>


The National Consumer Price Index System – SNIPC continuously and systematically produces the **[Broad National Consumer Price Index – IPCA](https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=o-que-e)**, which aims to measure the inflation of a set of products and services sold in retail, related to the personal consumption of families. This income range was created with the objective of guaranteeing coverage of 90% of families belonging to urban areas covered by the National System of Consumer Price Indexes - SNIPC.

This price index has as its collection unit commercial and service provision establishments, public service concessionaires and the internet and its collection extends, in general, from the 1st to the 30th of the reference month.

In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,5))
sns.lineplot(x=ifix.indices['date'] , linestyle="-", linewidth=10, y=(ifix.indices['IPCA']), color='purple')
clear()
plt.grid(True)
plt.show()

In [ ]:
ifix.indices[['date','IPCA']].tail()

#### CDI

<IMG SRC='https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/ANBIMA.svg/1200px-ANBIMA.svg.png'  STYLE='float:right; width:100px'/>


The CDI (Certificate of Interbank Deposit) is a very short-term security issued by banks. To some degree it resembles the **CBD**. This is because the **CDI** is also used by institutions to raise funds.

In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,5))
sns.lineplot(x=ifix.indices['date'] , linestyle="-", linewidth=10, y=(ifix.indices['CDI']), color='orange')
clear()
plt.grid(True)
plt.show()

In [ ]:
ifix.indices[['date','CDI']].tail()

#### IGP-M

<IMG SRC='https://logovtor.com/wp-content/uploads/2021/01/fgv-fundacao-getulio-vargas-logo-vector.png'  STYLE='float:right; width:100px'/>


The General Price Index - Market (IGP-M) is released monthly by the Brazilian Institute of Economics of the **[Getulio Vargas Foundation (FGV IBRE)](https://portal.fgv.br/noticias/igp-m-resultados-2021).** In this space, all information regarding the publications of this important indicator will be published.

In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,5))
sns.lineplot(x=ifix.indices['date'] , linestyle="-", linewidth=10, y=(ifix.indices['IGP-M']), color='lightblue')
clear()
plt.grid(True)
plt.show()

In [ ]:
ifix.indices[['date','IGP-M']].tail()

#### Daily return

IFIX x CDI x IPCA X SELIC x IGP-M

In [ ]:
plt.clf()
clear()
plt.figure(figsize=(30,15))
sns.lineplot(x=ifix.indices['date'] , linestyle="-",label="IFIX", linewidth=5, y=(ifix.indices['IFIX']), color='black')
sns.lineplot(x=ifix.indices['date'] , linestyle="-",label='IGP-M', linewidth=7, y=(ifix.indices['IGP-M']), color='green')
sns.lineplot(x=ifix.indices['date'] , linestyle="-",label="CDI", linewidth=13, y=(ifix.indices['CDI']), color='orange')
sns.lineplot(x=ifix.indices['date'] , linestyle="-",label="IPCA",linewidth=5, y=(ifix.indices['IPCA']), color='purple')
sns.lineplot(x=ifix.indices['date'] , linestyle="-",label="SELIC",linewidth=5, y=(ifix.indices['SELIC']), color='blue')

# defyning title and legend parameters
plt.title("Daily return | IFIX x CDI x IPCA X SELIC x IGP-M",size=25)
plt.legend(loc = "upper left"
           , frameon = True
           , fontsize = 22
           , ncol = 2 
           , fancybox = True
           , framealpha = 0.95
           , shadow = True
           , borderpad = 2);
clear()
plt.grid(True)
plt.show()

In [ ]:
ifix.indices

### continue...


thanks to [ @Wellington Silva](https://www.kaggle.com/gatunnopvp)

[Exploring Bovespa index(IBOV)](https://www.kaggle.com/gatunnopvp/exploring-bovespa-index-ibov-in-progress)

**data source by** https://infofundos.com.br/

<IMG SRC='https://i.pinimg.com/originals/c1/e2/31/c1e23101b8ae4e6536a6326b14dea06b.gif' 
     style="width:80px;float:right;" />